In [6]:
print('jmd shree ganesha')

jmd shree ganesha


In [7]:
from langchain.prompts import PromptTemplate
from langchain.chains.retrieval_qa.base import RetrievalQA
from langchain_pinecone import PineconeVectorStore
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import ctransformers
import os

In [8]:
os.getenv('PINECONE_API_KEY')
dataset = r"C:\Users\Admin\Desktop\Aster\FAQ Chatbot\FAQ-Chatbot\data"

In [9]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [10]:
extracted_data = load_pdf(dataset)

In [11]:
def text_splitter(data_pdf):
    splitter = RecursiveCharacterTextSplitter(chunk_size=150, chunk_overlap=10)
    chunks = splitter.split_documents(data_pdf)
    
    return chunks

In [12]:
text_split_chunks = text_splitter(extracted_data)

In [13]:
text_split_chunks

[Document(page_content='Yes, you can book appointments and many treatments directly with us. For most procedures, a \nGP referral is required.', metadata={'source': 'C:\\Users\\Admin\\Desktop\\Aster\\FAQ Chatbot\\FAQ-Chatbot\\data\\dataset.pdf', 'page': 0}),
 Document(page_content='If you are insured, we also recommend that you check with your insurer that you are covered to \nsee your preferred consultant at our hospital.', metadata={'source': 'C:\\Users\\Admin\\Desktop\\Aster\\FAQ Chatbot\\FAQ-Chatbot\\data\\dataset.pdf', 'page': 0}),
 Document(page_content='You can easily book an appointment online with many of our consultants. To find a consultant', metadata={'source': 'C:\\Users\\Admin\\Desktop\\Aster\\FAQ Chatbot\\FAQ-Chatbot\\data\\dataset.pdf', 'page': 0}),
 Document(page_content='near you, please clic k here. For more information, please call 0330 173 8200.', metadata={'source': 'C:\\Users\\Admin\\Desktop\\Aster\\FAQ Chatbot\\FAQ-Chatbot\\data\\dataset.pdf', 'page': 0}),
 Docu

In [14]:
print(len(text_split_chunks))

84


In [15]:
def text_to_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [16]:
embeddings = text_to_embeddings()

c:\Users\Admin\Desktop\Aster\FAQ Chatbot\FAQ-Chatbot\chatbot\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\Admin\Desktop\Aster\FAQ Chatbot\FAQ-Chatbot\chatbot\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [17]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [18]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [19]:
query_result

[-0.034477315843105316,
 0.03102317452430725,
 0.006734929047524929,
 0.026108937337994576,
 -0.03936201333999634,
 -0.16030247509479523,
 0.06692397594451904,
 -0.006441450212150812,
 -0.047450464218854904,
 0.014758866280317307,
 0.07087530195713043,
 0.055527571588754654,
 0.019193321466445923,
 -0.02625134214758873,
 -0.010109540075063705,
 -0.026940450072288513,
 0.022307438775897026,
 -0.022226622328162193,
 -0.1496925801038742,
 -0.017493009567260742,
 0.007676264736801386,
 0.05435226112604141,
 0.0032544350251555443,
 0.0317259319126606,
 -0.08462139964103699,
 -0.029405999928712845,
 0.05159561708569527,
 0.0481240451335907,
 -0.0033148531801998615,
 -0.05827915295958519,
 0.04196926951408386,
 0.022210637107491493,
 0.12818880379199982,
 -0.022338969632983208,
 -0.011656252667307854,
 0.06292839348316193,
 -0.032876309007406235,
 -0.09122602641582489,
 -0.031175294890999794,
 0.052699558436870575,
 0.04703482240438461,
 -0.0842030942440033,
 -0.030056163668632507,
 -0.020744

In [20]:
client = PineconeVectorStore(index_name = "chat",embedding=embeddings, pinecone_api_key='c5a1e2ec-7282-49f0-aed3-aa96eabfca32')

In [21]:
t_chunks = [t.page_content for t in text_split_chunks]
index_name = "chat"
doc_test = PineconeVectorStore.from_texts(t_chunks,
            index_name=index_name,
            embedding=embeddings
        )


In [22]:
doc_test

In [23]:
docsearch=PineconeVectorStore.from_existing_index(index_name, embeddings)

query = "How can I find out more about your consultants?"

docs=docsearch.similarity_search(query, k=5)

print("Result", docs)

Result [Document(page_content='please  click  here.  For more  information,  please  call 0330  173 8200.  \nQ. How  can I find out more  about  your consultants ?'), Document(page_content='please  click  here.  For more  information,  please  call 0330  173 8200.  \nQ. How  can I find out more  about  your consultants ?'), Document(page_content='please  click  here.  For more  information,  please  call 0330  173 8200.  \nQ. How  can I find out more  about  your consultants ?'), Document(page_content='You can easily  book  an appointment  online  with many  of our consultants.  To find a consultant  near you,'), Document(page_content='You can easily  book  an appointment  online  with many  of our consultants.  To find a consultant  near you,')]


In [24]:
prompt_template="""
Use the following pieces of information to answer the user's question. You are a helpful assistant who answers questions related to Nuffield Health Hospital.
If the question is not related to Nuffield Health, respond with "I don't know, It's out of my knowledge base.", 
else if you don't know the answer, just say that to contact the helpline number 0330 173 8200. Not to ask for the phone number.

Context: {context}

Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [25]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [26]:
llm=ctransformers.CTransformers(model=r"C:\Users\Admin\Desktop\Aster\FAQ Chatbot\FAQ-Chatbot\model\llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0})

In [27]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 5}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [28]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])

c:\Users\Admin\Desktop\Aster\FAQ Chatbot\FAQ-Chatbot\chatbot\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
